In [ ]:
import os
import sys

# Change to project root directory
os.chdir('/Users/mienhardt/Programming/babel_ai')
print(f"Current working directory: {os.getcwd()}")

# Add src to path for imports
sys.path.insert(0, 'src')


In [ ]:
from models.configs import ExperimentConfig

# Create ExperimentConfig copied from test_config.yaml
experiment_config = ExperimentConfig(
    fetcher_config={
        "fetcher": "sharegpt",
        "data_path": "data/human-ai_datasets/sharegpt_clean.json",
        "min_messages": 5,
        "max_messages": 20
    },
    analyzer_config={
        "analyzer": "similarity",
        "analyze_window": 5
    },
    agent_configs=[
        {
            "provider": "raven",
            "model": "llama3:70b",
            "temperature": 1.0,
            "max_tokens": None,
            "frequency_penalty": 0.0,
            "presence_penalty": 0.0,
            "top_p": 1.0,
        }
    ],
    agent_selection_method="round_robin",
    max_iterations=60,
    max_total_characters=1000000,
    output_dir="data/structured_analysis/single_agent_llama3_70b_gen_config_pivot"
)

print("ExperimentConfig created successfully!")
print(f"Fetcher: {experiment_config.fetcher_config.fetcher}")
print(f"Agent provider: {experiment_config.agent_configs[0].provider}")
print(f"Max iterations: {experiment_config.max_iterations}")


In [ ]:
# Create pivot dictionary for agent config settings
import numpy as np

# Define parameter ranges
temperature_values = np.arange(0.5, 1.55, 0.1)  # 0.0 to 1.5 in steps of 0.1
top_p_values = np.arange(0.5, 1.05, 0.1)  # 0.0 to 1.0 in steps of 0.1
penalty_values = np.arange(0.0, 2.1, 0.5)  # -2.0 to 2.0 in steps of 0.5

# Create the pivot dictionary
agent_config_pivot = {
    "temperature": temperature_values.tolist(),
    "top_p": top_p_values.tolist(),
    "frequency_penalty": penalty_values.tolist(),
    "presence_penalty": penalty_values.tolist()
}

print("Agent config pivot dictionary created:")
print(f"Temperature values ({len(agent_config_pivot['temperature'])} values):")
print(f"  {agent_config_pivot['temperature']}")
print(f"Top-p values ({len(agent_config_pivot['top_p'])} values):")
print(f"  {agent_config_pivot['top_p']}")
print(f"Frequency/Presence penalty values ({len(agent_config_pivot['frequency_penalty'])} values):")
print(f"  {agent_config_pivot['frequency_penalty']}")

# Total combinations
total_combinations = (len(agent_config_pivot['temperature']) * 
                     len(agent_config_pivot['top_p']) * 
                     len(agent_config_pivot['frequency_penalty']) * 
                     len(agent_config_pivot['presence_penalty']))
print(f"\nTotal possible combinations: {total_combinations}")


In [ ]:
import itertools
from copy import deepcopy

# Generate all combinations of agent config parameters
param_combinations = list(itertools.product(
    agent_config_pivot['temperature'],
    agent_config_pivot['top_p'], 
    agent_config_pivot['frequency_penalty'],
    agent_config_pivot['presence_penalty']
))

# Create ExperimentConfigs for each combination
experiment_configs = []
for i, (temp, top_p, freq_pen, pres_pen) in enumerate(param_combinations):
    config = deepcopy(experiment_config)
    config.agent_configs[0].temperature = temp
    config.agent_configs[0].top_p = top_p
    config.agent_configs[0].frequency_penalty = freq_pen
    config.agent_configs[0].presence_penalty = pres_pen
    config.output_dir = f"data/structured_analysis/single_agent_pivot_{i:04d}"
    experiment_configs.append(config)

def get_config_params(config):
    agent = config.agent_configs[0]
    return (agent.temperature, agent.top_p, 
            agent.frequency_penalty, agent.presence_penalty)

print(f"Created {len(experiment_configs)} ExperimentConfigs")
first = get_config_params(experiment_configs[0])
last = get_config_params(experiment_configs[-1])
print(f"First config: T={first[0]}, P={first[1]}, FP={first[2]}, PP={first[3]}")
print(f"Last config: T={last[0]}, P={last[1]}, FP={last[2]}, PP={last[3]}")


In [ ]:
from main import run_experiment_batch

# Run first 3 configurations
selected_configs = experiment_configs[:3]

print(f"Running {len(selected_configs)} experiments...")
for i, config in enumerate(selected_configs):
    print(f"Config {i}: T={config.agent_configs[0].temperature:.1f}, "
          f"P={config.agent_configs[0].top_p:.1f}, "
          f"FP={config.agent_configs[0].frequency_penalty:.1f}, "
          f"PP={config.agent_configs[0].presence_penalty:.1f}")

# Run experiments
await run_experiment_batch(selected_configs)
